# Projet de Machine Learning

In [1]:
from tools import *
import math as math
import copy
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [26]:
def gen_arti(centerx=1,centery=1,sigma=1,nbex=30,data_type=0,eps=0.01):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [29]:
def f_K(x, e, sigma=0.1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

# Calcul de tous les H(xl) pour le a minimal 
def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    f = np.zeros((0, 0))
    
    #print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = 0
        for j,e in enumerate(S):
            fl += a[j]*f_K(x,e)
        f = np.append(f, fl)
        
    #print "Done"
    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    #print "Calcul de H(xl)..."
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

# Calcul le a minimal approximé par Newton-R
def H(x, S, a_prec, dataset, y):

    a = a_optim(a_prec) 
    return HA(a, S, dataset, y), a

def a_optim(a_prec):

    f = K*a_prec 
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    d = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((d.shape[0], d.shape[0])))
    
    for i,element in enumerate(d):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    R = np.zeros((N,dataset.shape[1]))
    S = []
    predict = None
    lamda = 0.1
    
    Hk = 1
    Hk_1 = 0
    for i, e in enumerate(dataset):
        R[i] = e

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        if R.shape[0] == 0:
            break
            
        #print "Calcul des nouveaux H(xl)..."
        h = np.ones((R.shape[0], 1))
        S_b = []
        for e in S:
            S_b.append(e)
            
        A = []
        for i, xl in enumerate(R):
            S_b.append(xl)
            h[i], a = H(xl, S_b, a, dataset, y)
            A.append(a)
            S_b.pop()
            
            
        #print "Done"
        #print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        #print h
        #print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt

        #print "On l'ajoute a S..."

        S.append(R[xl_opt])
        #print "--------------> S:"

        #print "OK"
        #print "On l'enleve a R..."

        data = []
        for i, e in enumerate(R):
            if i == xl_opt:
                pass
            else:
                data.append(e)
        R = np.asarray(data)
        # update de a
        print "Updating a..."

        a = a_optim(A[xl_opt])
        #print "On garde celui associe au meilleur:", A[xl_opt]
        #print "OK"
        print "ecart", abs(Hk-Hk_1)/abs(Hk)
        
    print "S final"
    for el in S:
        print el
    print "Il y a ", len(S), " import points"
    
    print "a final"
    for e in a:
        print e
    return S, a

def predict(x, S, a):

    prediction = []
    for e in x:
        m = 0
        for i, s in enumerate(S):
            m += a[i]*f_K(e, s)
        
        if m < 0:
            prediction.append(-1)
        else:
            prediction.append(1)
            
    return prediction

def error(prediction, y):
    e = 0
    for i, p in enumerate(prediction):
        if p != y[i]:
            e = e+1
            
    return float(e)/float(len(y))
    


In [ ]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
S, a = ivm(data, y)
print "S final"
for el in S:
    print el
print "Il y a ", len(S), " import points"

print "a final"
for e in a:
    print e

prediction = predict(data, S, a)
print prediction
print y
erreur = error(prediction, y)
print "Erreur de classification", erreur

[[ 0.69353637  0.31873345]
 [-1.20755197 -1.43687398]
 [-0.86907445 -1.71055541]
 [ 0.63056712 -1.22555005]
 [-0.46516991 -0.86846528]
 [ 1.35945688  0.4558239 ]
 [ 0.5642716  -0.62418777]
 [-0.28415482 -2.50703988]
 [-0.49063705  1.22457862]
 [ 1.27804309  0.07382474]
 [ 0.05260528  0.65351183]
 [-0.98412916 -0.46593191]
 [ 0.84215898  1.00109883]
 [ 0.41241236  1.45639716]
 [ 0.74679517  1.6004318 ]
 [-1.46188364 -0.42342389]
 [-0.08141611 -2.81953496]
 [ 0.34843589  0.45021536]
 [-1.73164351 -0.58693753]
 [ 0.12770185  2.08455453]
 [ 0.55479908 -2.34399314]
 [ 0.25396915  0.0106597 ]
 [ 2.01971027  2.72686656]
 [ 1.50332971  0.77280472]
 [ 1.1694759   1.0900165 ]
 [-0.79374483 -0.74718215]
 [-0.84076651 -1.37978836]
 [ 0.97984623  0.16858524]
 [ 2.45757633  1.63776469]
 [-2.11347413 -0.55554764]]
Updating a...

In [25]:
data2, y2 = gen_arti()
data2 = np.asarray(data2)
prediction2 = predict(data2, S, a)
erreur2 = error(prediction2, y2)
print "Erreur de classification", erreur2

Erreur de classification 0.0
